In [1]:
import pandas as pd
import numpy as np
import datetime
import time 
import os
import gc
from tqdm import tqdm

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
ts = pd.read_hdf('./data/train.h5', 'xtrain')

In [5]:
ts.head()

,session_id,appeared,appeared_co,clickout item_ctn,clickout item_per,diff_mean,diff_mean_rel,diff_median,diff_median_rel,er_mean,...,action_type_click_rel_pos_avg,city_nunique,city_get_first,platform_get_first,device_get_first,nfilters_mean,nfilters_max,nfilters_min,nfilters_get_last,nimps_max
0,0266e4f67c9c9,2,0,2.0,1.000000,-2.956522,-0.026164,-3.0,-0.026549,NaN,...,0.619048,1,"Akaroa, New Zealand",NZ,desktop,NaN,NaN,NaN,NaN,23.0
1,0266e4f67c9c9,2,0,3.0,0.071429,-25.956522,-0.288406,-26.0,-0.288889,NaN,...,0.619048,1,"Akaroa, New Zealand",NZ,desktop,NaN,NaN,NaN,NaN,23.0
2,0266e4f67c9c9,2,0,1.0,0.052632,14.043478,0.108027,14.0,0.107692,NaN,...,0.619048,1,"Akaroa, New Zealand",NZ,desktop,NaN,NaN,NaN,NaN,23.0
3,0266e4f67c9c9,2,0,1.0,0.125000,14.043478,0.108027,14.0,0.107692,NaN,...,0.619048,1,"Akaroa, New Zealand",NZ,desktop,NaN,NaN,NaN,NaN,23.0
4,0266e4f67c9c9,2,0,3.0,0.272727,-12.956522,-0.125791,-13.0,-0.126214,NaN,...,0.619048,1,"Akaroa, New Zealand",NZ,desktop,NaN,NaN,NaN,NaN,23.0


In [9]:
ts.iloc[3][10:]

excellent_rating                                                                 1
good_rating                                                                      1
gr_mean                                                                        NaN
interaction item deals_ctn                                                       0
interaction item deals_per                                                       0
interaction item image_ctn                                                       7
interaction item image_per                                                   0.875
interaction item info_ctn                                                        0
interaction item info_per                                                        0
interaction item rating_ctn                                                      0
interaction item rating_per                                                      0
item_id                                                                    2176108
loca

In [ ]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# '''/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
#   return getattr(obj, method)(*args, **kwds)'''

In [ ]:
data_path = './data/'
!ls -lthr data/

In [ ]:
def pshape(df):
    print(f'df len: {df.shape[0]:,}')

In [ ]:
%%time
# nrows = 10000
nrows = None
train = pd.read_csv(data_path+'train.csv', nrows=nrows)

In [ ]:
xtrain = pd.read_hdf('./data/train.h5', 'xval')

In [ ]:
xtrain.head()

In [ ]:
set(xtrain.session_id).intersection(train[train.current_filters.notna()].session_id.unique())

In [ ]:
sid = np.random.choice(list(set(xtrain.session_id).intersection(train[train.current_filters.notna()].session_id.unique())), 1)[0]
sid

In [ ]:
train[train.session_id==sid]#.iloc[-1]

In [ ]:
xtrain[xtrain.session_id==sid]['n_matches']

In [ ]:
xtrain.n_matches.unique()

In [ ]:
meta

In [ ]:
train[train.session_id=='0001fe0007954']

In [ ]:
set(np.array([-1,0]))

In [ ]:
xtrain[xtrain.session_id=='0266e4f67c9c9']

In [ ]:
train[train.session_id=='0266e4f67c9c9'].iloc[:7]

In [ ]:
train[train.session_id=='0266e4f67c9c9'].iloc[2].impressions

In [ ]:
len(train[train.session_id=='0266e4f67c9c9'].iloc[2].impressions.split('|'))

In [ ]:
%%time
all_filters = train['current_filters'][train.current_filters.notna()].str.split('|').sum()

In [ ]:
%%time
all_filter = train['current_filters'][train.current_filters.notna()].str.split('|')

In [ ]:
all_filter.values

In [ ]:
%%time
a = [j for i in tqdm(all_filter) for j in i]

In [ ]:
type(np.nan) == float

In [ ]:
set(a)

In [ ]:
a_ctn = pd.value_counts(a)

In [ ]:
a_ctn['Gay Friendly']

In [ ]:
len(a)

In [ ]:
len(set(a))

In [ ]:
meta = pd.read_csv('./data/item_metadata.csv')

In [ ]:
ap = meta.properties.str.split('|')

In [ ]:
ap = [j for i in ap for j in i]

In [ ]:
ap

In [ ]:
len(set(ap))

In [ ]:
len(set(a).intersection(set(ap)))

In [ ]:
set(ap)

In [ ]:
set(ap) - set(a)

In [ ]:
set(a) - set(ap)

In [ ]:
xtrain.shape

In [ ]:
xtrain.columns

In [ ]:
train.current_filters.isna().sum()

In [ ]:
p = train.current_filters.str.split('|')

In [ ]:
p.iloc[-1]

In [ ]:
mp = pd.read_csv('./data/train_meta_fts.csv')

In [ ]:
mp['ps']

In [ ]:
test = pd.read_csv(data_path+'test.csv', nrows=nrows)

In [ ]:
train.head()

In [ ]:
%%time
train['ts'] = train['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))


In [ ]:
train_sids = train[train['ts'] <= datetime.datetime(2018, 11, 6)].session_id.unique()

In [ ]:
len(train_sids)

In [ ]:
train.session_id.nunique()

In [ ]:
749790/910683

In [ ]:
xtrain = pd.read_hdf('./data/train.h5', key='xtrain')

In [ ]:
xtrain.shape

In [ ]:
xval = pd.read_hdf('./data/train.h5', key='xval')

In [ ]:
xval.shape

In [ ]:
3311588/(3311588+15461026)

In [ ]:
pshape(train)

In [ ]:
test = pd.read_csv(data_path+'test.csv', nrows=None)
pshape(test)

In [ ]:
sub = pd.read_csv(data_path+'submission_popular.csv')

In [ ]:
sub.head()

In [ ]:
sids = train.session_id.unique()

In [ ]:
rid = np.random.choice(sids, 1)[0]
rid

In [ ]:
train[train.session_id==rid]

In [ ]:
train_users = train.user_id.unique()
test_users = test.user_id.unique()

In [ ]:
len(set(train_users).intersection(test_users))

In [ ]:
len(train_users)

In [ ]:
len(test_users)

In [ ]:
33614/250852

In [ ]:
xtrain.groupby('session_id').size().hist()

In [ ]:
pd.value_counts([2,2,2,10,-1])

In [ ]:
pd.value_counts([2,2,2,10,-1]).to_dict()

In [ ]:
# nunique previous clickout item_id
# previous appeared counts instead of boolean
# previous action_types occured

In [ ]:
train[train.session_id=='3e2a48ff1b59a']

In [ ]:
train.action_type.nunique()

In [ ]:
ty = train.action_type.unique()
m = {v: k for k, v in enumerate(ty)}

In [ ]:
train.iloc[:10]['action_type'].map(m).values

In [ ]:
train[train.session_id=='3e2a48ff1b59a'].iloc[[2, 3]].impressions.values

In [ ]:
a = np.eye(10)[np.random.randint(0, 5, 4)]
a

In [ ]:
a.sum(axis=0)

In [ ]:
np.tile(a.sum(axis=0), (3, 1))

In [ ]:
%%time
# encode action type info into reference ids, we do so by getting all the data from both train and test
action_train = pd.read_csv(data_path+'train.csv', usecols=['action_type', 'reference'])
action_test = pd.read_csv(data_path+'test.csv', usecols=['action_type', 'reference'])
action_cb = pd.concat([action_train, action_test], ignore_index=True)
del action_train, action_test
# remove rows whose action type is not ids
no_ids = ['search for poi', 'search for destination', 'filter selection', 'change of sort order']
action_cb = action_cb[~action_cb['action_type'].isin(no_ids)].reset_index(drop=True)
action_cb = action_cb.dropna().reset_index(drop=True)
is_id = action_cb['reference'].str.match('\d')
action_cb = action_cb[is_id].reset_index(drop=True)
action_cb['reference'] = action_cb['reference'].astype(int)

print('group on ids')
action_grp = action_cb.groupby('reference')['action_type']

In [ ]:
action_cb.reference

In [ ]:
action_ctn = action_grp.value_counts()

In [ ]:
action_ctn.head()

In [ ]:
actions = list(action_cb.action_type.unique())


In [ ]:
action_mapping = {v: k for k, v in enumerate(actions)}

In [ ]:
action_mapping

In [ ]:
action_ctn_df = action_ctn.reset_index(name='ctn')
action_ctn_df = action_ctn_df[action_ctn_df.index.str.match('\d')].reset_index(drop=True)

In [ ]:
action_ctn_df.head()

In [ ]:
action_ctn_df.shape

In [ ]:
action_ctn_df['action_id'] = action_ctn_df['action_type'].map(action_mapping)

In [ ]:
action_ctn_df.head()

In [ ]:
ohe = pd.DataFrame(np.eye(len(actions), dtype=int)[action_ctn_df.action_id.values], columns=actions)

In [ ]:
ohe.head()

In [ ]:
%%time
# (ohe*action_ctn_df['ctn'].values).head()
ohe = ohe.mul(action_ctn_df['ctn'], axis=0)

In [ ]:
action_ctn_df = pd.concat([action_ctn_df, ohe], axis=1)

In [ ]:
action_ctn_df.head()

In [ ]:
action_ctn_df.head()

In [ ]:
action_encoding = action_ctn_df.groupby('reference')[actions].sum()
# action_ctn_df.set_index('reference').groupby(level=0).sum()
# action_encoding[actions]/action_encoding.sum(axis=1)
normalized = action_encoding.div(action_encoding.sum(axis=1), axis=0)
normalized.head()

In [ ]:
action_encoding = action_encoding.join(normalized, lsuffix='_ctn', rsuffix='_per',)

In [ ]:
action_encoding.head()

In [ ]:
action_encoding.reset_index()

In [ ]:
action_encoding.index.str.match('\d')

## NOTES
* the order of filter selection and what it selects is probably important, as it unrolls what the customer wants, maybe the order is not than important
    change of sort order add on top of this

## TO DOS:
* show feature importance

In [ ]:
train[train.action_type=='filter selection'].reference.nunique()

### clip sessions off to last click

In [ ]:
%%time
# get all rows upto the last clickout action (some rows has reference but it's not clickout action)
def up_to_last_click(grp):
    check = grp.action_type == 'clickout item'
    if check.sum() != 0:
        return grp.iloc[:np.argwhere(check)[-1][0]+1]
    else:
        return grp
        
train = train.groupby('session_id').apply(up_to_last_click).reset_index(drop=True)

In [ ]:
%%time
# get utc timestamp
train['ts'] = train['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))

In [ ]:
%%time
# get utc timestamp
test['ts'] = test['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))

In [ ]:
train.ts.hist(bins=20, xrot=90)

In [ ]:
test.ts.hist(bins=20, xrot=90)

In [ ]:
test.ts.head()

In [ ]:
%%time
# only look at sessions with clickouts (for now)
# first filter out sessions that does not have a click-out
def check_clickout(grp):
     # sessions has clickouts
    has_clickout = 'clickout item' in grp['action_type'].unique()
    # last row has reference and it's not nan
    has_ref = ((grp['action_type'].iloc[-1] == 'clickout item') & 
               (grp.iloc[-1][['impressions', 'reference', 'prices']].isna().sum()==0))
    return has_clickout & has_ref
    
clicked = train.groupby('session_id').apply(check_clickout)
click_session_ids = clicked[clicked].index
# filter
train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

In [ ]:
# %%time
# # only look at sessions with clickouts (for now)
# # first filter out sessions that does not have a click-out
# def check_clickout(grp):
#      # sessions has clickouts
#     has_clickout = 'clickout item' in grp['action_type'].unique()
#     # last row has reference and it's not nan
#     has_ref = ((grp['action_type'].iloc[-1] == 'clickout item') & 
#                (pd.notnull(grp.iloc[-1]['impressions'])) &
#                (pd.notnull(grp.iloc[-1]['reference'])) &
#                (pd.notnull(grp.iloc[-1]['prices'])))
#     return has_clickout & has_ref
    
# clicked = train.groupby('session_id').apply(check_clickout)
# click_session_ids = clicked[clicked].index
# # filter
# train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

In [ ]:
pshape(train)

In [ ]:
# d = train.groupby('session_id')
# for i, j in d:
#     print(j.head())
#     print('='*20)
#      # sessions has clickouts
#     has_clickout = 'clickout item' in j['action_type'].unique()
#     print(has_clickout)
#     # last row has reference and it's not nan
#     has_ref = ((j['action_type'].iloc[-1] == 'clickout item') & 
#                (pd.notnull(j.iloc[-1]['impressions'])) &
#                (pd.notnull(j.iloc[-1]['reference'])))
#     print(has_ref)
#     break

### meta info

In [ ]:
%%time
meta = pd.read_csv(data_path+'item_metadata.csv', nrows=nrows)
meta['properties'] = meta['properties'].str.lower()

In [ ]:
# %%time
# # checkout properties of items
# def get_property_count(row):
#     return len(row.split('|'))



# item_p_ctn = meta['properties'].apply(get_property_count)
# item_p_ctn.describe()

In [ ]:
%%time
def get_property(row):
    return row.lower().split('|')
# meta['ps'] = meta['properties'].apply(get_property)

meta['ps'] = meta['properties'].str.split('|')
# numer of properties
meta['nprop'] = meta.ps.str.len()
# star ratings
meta['star'] = meta.properties.str.extract('[\|](\d) star')
meta['star'] = meta['star'].astype(float)

In [ ]:
%%time
# # create rating columns
# ratings = ['good rating', 'satisfactory rating', 'excellent rating']
# for r in ratings:
#     meta[r.replace(' ', '_')] = meta.properties.str.findall(f'\|{r}').str.len()
    
    
def create_rating_colum(meta):
    # create rating columns
    ratings = ['good rating', 'satisfactory rating', 'excellent rating']
    for r in ratings:
        meta[r.replace(' ', '_')] = meta.properties.str.findall(f'\|{r}').str.len()

In [ ]:
def get_popularity(df):
    """Get number of clicks that each item received in the df."""

    mask = df["action_type"] == "clickout item"
    df_clicks = df[mask]
    df_item_clicks = (df_clicks
                      .groupby("reference")
                      .size()
                      .reset_index(name="n_clicks")
                      .transform(lambda x: x.astype(int)))
    return df_item_clicks

In [ ]:
%%time
item_popularity = get_popularity(train)
meta = pd.merge(meta, item_popularity, left_on='item_id', right_on='reference')

In [ ]:
meta.head()

In [ ]:
meta = (meta[['item_id', 'nprop', 'n_clicks', 'star', 'good_rating', 'satisfactory_rating', 'excellent_rating']]
        .set_index('item_id'))

### Create session features
---

In [ ]:
# %%time
# # find out if same customers have multiple sessions
# s = train.drop_duplicates(subset=['user_id', 'session_id'])
# s.session_id.duplicated().sum()
# # seems like not

In [ ]:
# train.current_filters[:1000].unique()

In [ ]:
%%time
# def get_filters(x):
#     if type(x) == str or type(x) == list:
#         return x.split('|')
#     else:
#         return np.nan
    
# def get_impressions(x):
#     if type(x) == str:
#         return x.split('|')
#     else:
#         return np.nan
    
    
train['filters'] = train.current_filters.str.split('|')
train['nfilters'] = train.filters.str.len()
train['imps_list'] = train.impressions.str.split('|')
nn_mask = train['imps_list'].notnull()
train.loc[nn_mask, 'imps_list'] = train.loc[nn_mask, 'imps_list'].apply(lambda x: [int(i) for i in x])
train['nimps'] = train.imps_list.str.len()

In [ ]:
# train.nimps.describe()

In [ ]:
# # no switch of devices during session
# (train.groupby('session_id')['device'].nunique()!=1).sum()

In [ ]:
# # differnet city during session
# (train.groupby('session_id')['city'].nunique()!=1).sum()

In [ ]:
%%time
# some custom funcs used in agggregation
def mean_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.mean(np.diff(np.sort(x)))
    
def var_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.var(np.diff(np.sort(x)))
    
def get_first(x):
    return x.iloc[0]

def get_last(x):
    return x.iloc[-1]

def n_clickouts(x):
    return (x=='clickout item').sum()

def click_rel_pos_avg(x):
    return np.mean(np.argwhere((x=='clickout item')))/len(x)

def ptp(x):
    return x.max() - x.min()


# define some aggs
# session_aggs = {'timestamp': [np.ptp, mean_dwell_time, var_dwell_time],
session_aggs = {'timestamp': [ptp, mean_dwell_time, var_dwell_time],
                'step': ['max'],
                'action_type': ['nunique', n_clickouts, click_rel_pos_avg],
                'city': ['nunique', get_first],
                'platform': [get_first],
                'device': [get_first],
                'nfilters': ['mean', 'max', 'min', get_last],
                'nimps': ['max']
               }

In [ ]:
session_grp = train.groupby('session_id')

In [ ]:
%%time
# import os.path
session_file = 'session_fts.csv'
if os.path.isfile(session_file):
    session_fts = pd.read_csv(session_file)
else:
    session_fts = session_grp.agg(session_aggs)
    session_fts.columns = ['_'.join(col).strip() for col in session_fts.columns.values]
    session_fts.to_csv(session_file)

In [ ]:
# session_fts.columns.values
session_fts.head()

In [ ]:
# train[train.session_id=='62991f7c78f27']

### Create clickout features
---

In [ ]:
import multiprocessing as mp
from functools import partial


def get_session_item_pairs(args):
    # grab the args
    gids, session_df, meta_df = args
    # selecting the assigned session ids and grouping on session level
    grps = (session_df[session_df['session_id'].isin(gids)]
            .reset_index(drop=True)
            .groupby('session_id'))
    
    # use apply to compute session level features
    session_compute_func = partial(compute_session_item_pair, meta_df=meta_df)
    session_features = grps.apply(session_compute_func)
    
    return session_features
        
    
# def compute_session_item_pair(session_df, g_id, buy_df):
def compute_session_item_pair(session_df, meta_df):
    sdf = session_df.copy()
    last_row = sdf.iloc[-1]
    above = sdf.iloc[:-1]
    # get previous appeard impressions
    prev = above[above['impressions'].notnull()]
    prev_imps = prev['imps_list']
    unique_imps = [j for i in prev_imps for j in i]

    imp_l = last_row['imps_list']
    prices = last_row['prices'].split('|')
    prices = [int(p) for p in prices]
    # whether the impression appeared before
    appeared = [int(i in unique_imps) for i in imp_l]
    # the location of the impression
    locs = list(range(len(imp_l)))

    # build the df
    result = pd.DataFrame({'appeared': appeared, 'location': locs, 'price': prices}, index=imp_l)
    result.index.name = 'item_id'
    price_ind = np.argsort(result['price'].values) + 1
    result['rel_price_rank'] = price_ind/len(imp_l)
#     result['rel_price_rank'] = result[['location', 'price']].sort_values(by='price')['location']/len(imp_l)

    result['price_mean'] = np.mean(result['price'])
    result['price_median'] = np.median(result['price'])

    result_price = result['price'].values
    result_price_mean = result['price_mean'].values 
    result_price_median = result['price_median'].values

    result['diff_mean'] = result_price - result_price_mean
    result['diff_median'] = result_price - result_price_median
    result['diff_mean_rel'] = (result_price - result_price_mean)/result_price
    result['diff_median_rel'] = (result_price - result_price_median)/result_price

    # fetch the meta data
    result = result.join(meta_df, on='item_id')
    result['p_mean'] = np.mean(result['n_clicks'].values)
    result['star_mean'] = np.mean(result['star'].values)
    result['gr_mean'] = np.mean(result['good_rating'].values)
    result['sr_mean'] = np.mean(result['satisfactory_rating'].values)
    result['er_mean'] = np.mean(result['excellent_rating'].values)
    result.reset_index(inplace=True)

    # get target
    ref = int(last_row['reference'])
    result['target'] = (result['item_id'].values == ref).astype(int)
    return result


# 2) dwell time for each clickout
# 3) the rating and star and nproperty -> mean and median for 
#     the other impressions in list (or the quantile of these and prices)
# 4) location and relative location in the impressions list


def generate_session_item_pairs(sessions_df, meta_df, nprocs=None):
    t1 = time.time()
    if nprocs is None:
        nprocs = mp.cpu_count() - 1 
        print('Using {} cores'.format(nprocs))

    sids = sessions_df.session_id.unique()
    
    pairs = []
    # create iterator to pass in args
    def args_gen():
        for i in range(nprocs):
            yield (sids[range(i, len(sids), nprocs)], sessions_df, meta_df)
    
    # init multiprocessing pool
    pool = mp.Pool(nprocs)
    for pair in pool.map(get_session_item_pairs, args_gen()):
        pairs.append(pair)
    pool.close()
    pool.join()
    print('Done genearting, total time took: {0:.2f}mins'.format((time.time()-t1)/60))

    return pd.concat(pairs, axis=0)#, ignore_index=True)

In [ ]:
# %%time
# result_df = generate_session_item_pairs(train[train.session_id.isin(sids)], meta, nprocs=None)

In [ ]:
# %%time
# result_df = generate_session_item_pairs(train[:10000], meta, nprocs=None)

In [ ]:
%%time
result_df = generate_session_item_pairs(train, meta, nprocs=None)

In [ ]:
result_df.to_csv('./data/result.csv')

In [ ]:
result_df.shape

In [ ]:
meta.shape

In [ ]:
result_df.head()

In [ ]:
%%time
result_df = result_df.reset_index(level='session_id')

In [ ]:
%%time
# join on session features
result_df = result_df.set_index('session_id')

In [ ]:
%%time
final = result_df.join(session_fts)

In [ ]:
final.to_csv('./data/final.csv')

to do:

1) check the time range of train vs test

In [ ]:
final.head()

In [ ]:
train.ts.hist(bins=20)

In [ ]:
(train.ts <= datetime.datetime(2018,11,6)).sum()

In [ ]:
(train.ts > datetime.datetime(2018,11,6)).sum()

In [ ]:
train_sids = train[train.ts <= datetime.datetime(2018,11,6)].session_id.unique()

In [ ]:
final.head()

In [ ]:
# final.reset_index(name='session_id').head()
xtrain = final[final.index.isin(train_sids)]

In [ ]:
xval = final[~final.index.isin(train_sids)]

In [ ]:
xtrain.to_hdf('./data/train.h5', key='xtrain', mode='w')

In [ ]:
xtrain.to_hdf('./data/train.h5', key='xval', mode='a')